# Лабораторная работа 2. Модели представления текстов

In [2]:
import json, time, os, string, re, copy, math
import numpy as np
import pandas as pd
import pymorphy2
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer

In [3]:
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\artem\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\artem\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [4]:
ENCODING_DEFAULT = 'utf-8'
TRAIN_FILE = 'train.json'
TEST_FILE = 'test.json'
PRECISION = 4 # the number of signs after the comma

ID = 'id'
TEXT = 'text'
SENTIMENT = 'sentiment'
SENTIMENTS = ['negative', 'positive', 'neutral']
POSITIVE = 0
NEGATIVE = 1
NEUTRAL = 2

ORIGIN_WORD = 'OriginWord'
NORMAL_FORM = 'NormalForm'
POS = 'POS'
CASE = 'Case'
GENDER = 'Gender'
DICT_EXIST = 'FromDict'
WORDS = "Words"
FREQ = "Freq"
UNIQ = "uniquies"
COUNT = "count"

In [5]:
gr = {
    "NOUN":"имя существительное",
    "ADJF": "имя прилагательное (полное)",
    "NOUN":"имя существительное",
    "ADJF":"имя прилагательное (полное)",
    "ADJS":"имя прилагательное (краткое)",
    "COMP":"компаратив",
    "VERB":"глагол (личная форма)",
    "INFN":"глагол (инфинитив)",
    "PRTF":"причастие (полное)",
    "PRTS":"причастие (краткое)",
    "GRND":"деепричастие",
    "NUMR":"числительное",
    "ADVB":"наречие",
    "NPRO":"местоимение-существительное",
    "PRED":"предикатив",
    "PREP":"предлог",
    "CONJ":"союз",
    "PRCL":"частица",
    "INTJ":"междометие",
    "nomn":"именительный",
    "gent":"родительный",
    "datv":"дательный",
    "accs":"винительный",
    "ablt":"творительный",
    "loct":"предложный",
    "voct":"звательный",
    "gen2":"второй родительный (частичный)",
    "acc2":"второй винительный",
    "loc2":"второй предложный (местный)",
    "masc":"мужской род",
    "femn":"женский род",
    "neut":"средний род",
    "LATN":"Токен состоит из латинских букв (например, “foo-bar” или “Maßstab”)",
    "PNCT":"Пунктуация (например, , или !? или …)",
    "NUMB":"Число (например, “204” или “3.14”)",
    "intg":"целое число (например, “204”)",
    "real":"вещественное число (например, “3.14”)",
    "ROMN":"Римское число (например, XI)",
    "UNKN":"Токен не удалось разобрать",
}

### Задание 1. Подготовка текстового корпуса
- Прочитайте описание задачи Sentiment analysis: https://www.kaggle.com/c/sentiment-analysis-in-russian.
- Скачайте файлы `train.json` и `test.json`: https://www.kaggle.com/c/sentiment-analysis-in-russian/data.
- Загрузите в ноутбук скачанные файлы, выведите информацию по ним.

In [6]:
def read_json(filename):
    start = time.perf_counter()
    with open(filename, 'r', encoding=ENCODING_DEFAULT) as file:
        data = json.load(file)
    print("[INFO]: read time: %0.4f sec" % (time.perf_counter() - start))
    return data

In [7]:
def print_file_description(data, train=False):
    '''размер файла, количество текстов, средняя/макс./мин. длина текста, распределение тональности для обучающей выборки'''
    text_lens = []
    if train:
        sentiments = dict.fromkeys(SENTIMENTS, 0)
    for news in data:
        if train:
            sentiments[news[SENTIMENT]] += 1
        text_lens.append(len(news[TEXT]))
    print("Размер(байты):", os.path.getsize(TRAIN_FILE if train else TEST_FILE))
    print("Количество примеров:", len(data))
    print("Средняя длина текста:", np.mean(text_lens))
    print("Максимальная длина текста:", np.max(text_lens))
    print("Минимальная длина текста:", np.min(text_lens))
    if train:
        print("Распределение тональностей:")
        print("\tПозитивных:", sentiments[SENTIMENTS[POSITIVE]])
        print("\tНегативных:", sentiments[SENTIMENTS[NEGATIVE]])
        print("\tНейтральных:", sentiments[SENTIMENTS[NEUTRAL]])

In [8]:
train_json = read_json(TRAIN_FILE)
print_file_description(train_json, train=True)

[INFO]: read time: 0.4575 sec
Размер(байты): 59298269
Количество примеров: 8263
Средняя длина текста: 3911.85017548106
Максимальная длина текста: 381498
Минимальная длина текста: 28
Распределение тональностей:
	Позитивных: 1434
	Негативных: 2795
	Нейтральных: 4034


In [9]:
test_json = read_json(TEST_FILE)
print_file_description(test_json)

[INFO]: read time: 0.1556 sec
Размер(байты): 15417058
Количество примеров: 2056
Средняя длина текста: 4102.4143968871595
Максимальная длина текста: 320754
Минимальная длина текста: 35


### Задание 2. Булевская модель
Напишите функцию `get_bool_model()` на входе которой текстовый корпус, на выходе – булевская матрица термин-документ.

В функции должны осуществляться следующие действия:
- все слова преобразуются в нормальную форму;
- удаляются стоп-слова;
- удаляются (или оставляются) слова заданных частей речи (список частей речи должен передаваться в виде параметра);
- удаляются слова, частотность которых во всем корпусе ниже заданного порога (параметр);
- создается словарь корпуса.

Нельзя использовать библиотечные функции `scikit-learn`.  
Можно использовать функции, разработанные в этой и предыдущей лабораторных работах.

In [10]:
class SetEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, set):
            return list(obj)
        return super().default(self, obj)

In [11]:
# tools
morph = pymorphy2.MorphAnalyzer()
nlp = word_tokenize

stopwords_dict = Counter(stopwords.words('russian')) # {"sw1" : 1, "sw2" : 1, ..., "swN" : 1}
re_numbers_pattern = re.compile(r"\d+([,\.]\d+){0,}") # https://regex101.com/r/1V0HDV/1
re_spaces_pattern = re.compile(r"\s{2,}") # two or more spaces
re_newline_pattern = re.compile(r"\n") # newline

def save_data(data, filename='tmp'):
    with open(f"{filename}.json", "w+", encoding='utf-8') as f:
        tojs = json.dumps(data, indent=4, ensure_ascii=False, cls=SetEncoder)
        f.write(tojs)

def get_data(t):
    return 'None' if t is None else t

def check_word(word):
    # token filter
    if word not in string.punctuation and \
        len(re.findall(re_numbers_pattern, word)) == 0:
        return True
    return False

def text_prepare(text):
    # replace '\n' on ' ', '<spaces>' on '<space>'
    text = re.sub(re_newline_pattern, ' ', text)
    text = re.sub(re_spaces_pattern, ' ', text)
    return text

def parse_text(text):
    ret = []
    tokens = nlp(text_prepare(text))
    for word in tokens:
        if check_word(word):
            parsed = morph.parse(word)[0]
            data = {
                ORIGIN_WORD : word,
                NORMAL_FORM : parsed.normal_form,
                POS         : parsed.tag.POS if parsed.tag.POS is not None else 'NOUN',
                CASE        : parsed.tag.case,
                GENDER      : parsed.tag.gender,
                DICT_EXIST  : parsed.is_known,
            }
            ret.append(data)              
    return ret

def get_dictionary(parsed):
    tmp = {}
    for value in parsed:
        if tmp.get(value[NORMAL_FORM]) is None:
            tmp.update({
            value[NORMAL_FORM] : { 
                POS   : value[POS], 
                WORDS : set(([value[ORIGIN_WORD]])),
                FREQ : 1,
            }})
        else:
            tmp[value[NORMAL_FORM]][WORDS].add(value[ORIGIN_WORD])
            tmp[value[NORMAL_FORM]][FREQ] += 1  
    return tmp

def clearing(data, freq_rm, pos_rm):
    tmp = {}
    print("[INFO]: lenght of corpus dictionary befor clearing:", len(data))
    for key, val in data.items():
        if not (val[POS] in pos_rm or val[FREQ] < freq_rm or key in stopwords_dict):
            tmp.update({key : val})
    print("[INFO]: lenght of corpus dictionary after clearing:", len(tmp))
    return tmp

In [12]:
s = 'aaa ff v    ffg\nsdfsf fff \n   fsfsfsf fsd g 123 fsdf'
text_prepare(s)

'aaa ff v ffg sdfsf fff fsfsfsf fsd g 123 fsdf'

In [13]:
def get_bool_model(corpus, freq_remove=0, pos_remove=[]):
    parsed_texts = [] # to store the parsed texts
    documents_dicts = [] # list of dicts of texts

    # loop by corpus elements
    for text in corpus:
        parsed = parse_text(text[TEXT]) # text morph analysis
        parsed_texts.extend(parsed) # adding to list of all parsed texts
        documents_dicts.append(get_dictionary(parsed)) # forming dictionary of text and adding to list
    corpus_dict = get_dictionary(parsed_texts)
    
    #save_data(corpus_dict) #todo delete

    # clearing the text from stopwords, low-frequency and exclude some pos
    corpus_dict_cleared = clearing(corpus_dict, freq_remove, pos_remove)

    # forming term matrix
    term_matrix = []
    for word in corpus_dict_cleared:
        word_exist = []
        for doc_dict in documents_dicts:
            word_exist.append(True if word in doc_dict else False)
        term_matrix.append(word_exist)
        
    return term_matrix, corpus_dict_cleared

In [14]:
bool_model, dict_cleared = get_bool_model(train_json[:3], freq_remove=1, pos_remove=['NOUN'])
df = pd.DataFrame(bool_model, index=dict_cleared.keys())
df.to_csv("bool_model.csv")
print("\nМатрица термин-документ")
df

[INFO]: lenght of corpus dictionary befor clearing: 662
[INFO]: lenght of corpus dictionary after clearing: 294

Матрица термин-документ


,0,1,2
досудебный,True,True,False
начать,True,True,False
национальный,True,False,False
сообщить,True,True,False
финансовый,True,False,False
...,...,...,...
формировать,False,False,True
сохраниться,False,False,True
модельный,False,False,True
зависеть,False,False,True


### Задание 3. Модель TF-IDF
Напишите функцию `get_tfidf_model()` на входе которой текстовый корпус, на выходе – матрица термин-документ c TF-IDF-весами.

В функции должны осуществляться следующие действия:
- все слова преобразуются в нормальную форму;
- удаляются стоп-слова;
- удаляются (или оставляются) слова заданных частей речи (список частей речи должен передаваться в виде параметра);
- удаляются слова, частотность которых во всем корпусе ниже заданного порога (параметр);
- создается словарь корпуса;
- вычисляются глобальные IDF-веса и сохраняются в словарь;
- слова для документов взвешиваются в соответствии со схемой TF-IDF.

Нельзя использовать библиотечные функции `scikit-learn`.  
Можно использовать функции, разработанные в предыдущей лабораторной работе.

In [43]:
def compute_tfidf(documents_dict):

    def compute_tf(document):
        tf_text = {k : v[FREQ] for k, v in document.items()}
        for word in tf_text:
            tf_text[word] = tf_text[word] / float(len(document))
        return tf_text

    def compute_idf(word, documents_dict):
        return np.log(1+(len(documents_dict) / sum([1.0 for doc in documents_dict if word in doc])))

    documents_list = []
    for document in documents_dict:
        tf_idf_dictionary = {}
        computed_tf = compute_tf(document)
        for word in computed_tf:
            tf_idf_dictionary[word] = computed_tf[word] * compute_idf(word, documents_dict)
        documents_list.append(tf_idf_dictionary)

    return documents_list

In [64]:
# начало такое же
def get_tfidf_model(corpus, freq_remove=0, pos_remove=[]):
    parsed_texts = [] # to store the parsed texts
    documents_dicts = [] # list of dicts of texts

    # loop by corpus elements
    for text in corpus:
        parsed = parse_text(text[TEXT]) # text morph analysis
        parsed_texts.extend(parsed) # adding to list of all parsed texts
        documents_dicts.append(clearing(get_dictionary(parsed), freq_remove, pos_remove)) # forming dictionary of text and adding to list
    corpus_dict = get_dictionary(parsed_texts)
    
    # clearing the text from stopwords, low-frequency and exclude some pos
    corpus_dict_cleared = clearing(corpus_dict, freq_remove, pos_remove)

    tfidf_dict = compute_tfidf(documents_dicts)

    texts = []
    for clear_document_dict in documents_dicts:
      tmp_text = []
      for word in clear_document_dict:
        for _ in range(clear_document_dict[word][FREQ]):
          tmp_text.append(word)
      texts.append(' '.join(tmp_text))

      #norm L2
      tfidf_dict_norm = []
      for tfidf_doc in tfidf_dict:
        sum_of_squares = sum([x**2 for x in tfidf_doc.values()])
        for w, f in tfidf_doc.items():
          tfidf_doc[w] /= math.sqrt(sum_of_squares)
        tfidf_dict_norm.append(tfidf_doc)
        
        
    return tfidf_dict_norm, corpus_dict_cleared, texts

Постройте матрицу термин-документ для текстовых корпусов из первого задания.

In [65]:
tfidf_model, corpus_dict, texts = get_tfidf_model(train_json[:4], freq_remove=0, pos_remove=[])
df = pd.DataFrame(tfidf_model).fillna(0)
df.to_csv("tfidf_model.csv")
df.T[0:10]

[INFO]: lenght of corpus dictionary befor clearing: 110
[INFO]: lenght of corpus dictionary after clearing: 94
[INFO]: lenght of corpus dictionary befor clearing: 135
[INFO]: lenght of corpus dictionary after clearing: 119
[INFO]: lenght of corpus dictionary befor clearing: 489
[INFO]: lenght of corpus dictionary after clearing: 437
[INFO]: lenght of corpus dictionary befor clearing: 211
[INFO]: lenght of corpus dictionary after clearing: 192
[INFO]: lenght of corpus dictionary befor clearing: 800
[INFO]: lenght of corpus dictionary after clearing: 744


,0,1,2,3
досудебный,0.096895,0.043312,0.000000,0.000000
расследование,0.096895,0.043312,0.000000,0.000000
факт,0.074730,0.033404,0.008637,0.000000
покупка,0.048448,0.000000,0.022397,0.000000
енпф,0.283898,0.000000,0.000000,0.000000
пакет,0.070975,0.000000,0.000000,0.000000
облигация,0.070975,0.000000,0.000000,0.000000
тоо,0.048448,0.000000,0.000000,0.078577
``,0.283898,0.000000,0.000000,0.000000
бузгул,0.070975,0.000000,0.000000,0.000000


Сравните результаты работы вашей функции (полученные веса) с результатами работы класса [TfidfVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html) из `scikit-learn`.

In [46]:
vectorizer = TfidfVectorizer(smooth_idf = False, tokenizer = word_tokenize, vocabulary = corpus_dict.keys())
X = vectorizer.fit_transform(texts)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:1448: RuntimeWarning: divide by zero encountered in true_divide
  idf = np.log(n_samples / df) + 1


In [58]:
data = X.toarray().T
df = pd.DataFrame(data=data, index=list(corpus_dict.keys()))
df[:10]

,0,1,2,3
досудебный,0.093068,0.044753,0.000000,0.000000
расследование,0.093068,0.044753,0.000000,0.000000
факт,0.070781,0.034036,0.008775,0.000000
покупка,0.046534,0.000000,0.023076,0.000000
енпф,0.262338,0.000000,0.000000,0.000000
пакет,0.065584,0.000000,0.000000,0.000000
облигация,0.065584,0.000000,0.000000,0.000000
тоо,0.046534,0.000000,0.000000,0.077358
``,0.524676,0.000000,0.000000,0.000000
бузгул,0.065584,0.000000,0.000000,0.000000


### Задание 4. Модель word2vec
Напишите функцию `get_word2vec_model()` на входе которой текстовый корпус и модель `word2vec`, на выходе – матрица термин-документ c word2vec-весами.

В функции должны осуществляться следующие действия:
- все слова преобразуются в нормальную форму (при необходимости – в зависимости от используемой модели);
- удаляются стоп-слова;
- удаляются (или оставляются) слова заданных частей речи (список частей речи должен передаваться в виде параметра);
- удаляются слова, частотность которых во всем корпусе ниже заданного порога (параметр);
- создается словарь корпуса;
- вычисляются веса word2vec для заданной модели.


Можно использовать библиотечные функции и функции, разработанные в лабораторных работах.

In [ ]:
def get_word2vec_model():
    # Ваш код здесь
    return

Постройте матрицу термин-документ для текстовых корпусов из первого задания.

In [ ]:
# Ваш код здесь